In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
train_df= pd.read_csv("/Users/andretomaz/code/XavierLooyens/Churn_prediction/raw_data/train_df_2.csv")

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636730 entries, 0 to 636729
Data columns (total 38 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  636730 non-null  int64  
 1   msno                        636730 non-null  object 
 2   is_churn                    636730 non-null  int64  
 3   payment_method_id           636730 non-null  float64
 4   payment_plan_days           636730 non-null  float64
 5   plan_list_price             636730 non-null  float64
 6   actual_amount_paid          636730 non-null  float64
 7   is_auto_renew               636730 non-null  float64
 8   is_cancel                   636730 non-null  float64
 9   remaining_plan_duration     636730 non-null  int64  
 10  is_discount                 636730 non-null  int64  
 11  num_25                      636730 non-null  float64
 12  num_50                      636730 non-null  float64
 13  num_75        

In [4]:
train_df = train_df.drop(['Unnamed: 0', 'usage_from_ltd', 'msno','discount_percentage'], axis=1)

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636730 entries, 0 to 636729
Data columns (total 34 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   is_churn                    636730 non-null  int64  
 1   payment_method_id           636730 non-null  float64
 2   payment_plan_days           636730 non-null  float64
 3   plan_list_price             636730 non-null  float64
 4   actual_amount_paid          636730 non-null  float64
 5   is_auto_renew               636730 non-null  float64
 6   is_cancel                   636730 non-null  float64
 7   remaining_plan_duration     636730 non-null  int64  
 8   is_discount                 636730 non-null  int64  
 9   num_25                      636730 non-null  float64
 10  num_50                      636730 non-null  float64
 11  num_75                      636730 non-null  float64
 12  num_985                     636730 non-null  float64
 13  num_100       

In [6]:
scale_features = ['payment_method_id',
                  'payment_plan_days',
                  'plan_list_price',
                  'actual_amount_paid',
                  'remaining_plan_duration',
                  'num_25',
                  'num_50',
                  'num_75',
                  'num_985',
                  'num_100',
                  'num_unq',
                  'total_secs',
                  'city',
                  'bd',
                  'registered_via']

In [7]:
# Build the pipeline with the different steps
num_pipeline = make_pipeline(SimpleImputer(strategy="mean"),StandardScaler())

preprocessor = ColumnTransformer([
    ('num_transformer', num_pipeline, scale_features)],
    remainder='passthrough'
)


In [8]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_transformer',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['payment_method_id', 'payment_plan_days',
                                  'plan_list_price', 'actual_amount_paid',
                                  'remaining_plan_duration', 'num_25', 'num_50',
                                  'num_75', 'num_985', 'num_100', 'num_unq',
                                  'total_secs', 'city', 'bd',
                                  'registered_via'])])

In [9]:
X = train_df.drop(['is_churn'], axis=1)
y= train_df['is_churn']

In [10]:
preprocessor.fit(X)
            

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_transformer',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['payment_method_id', 'payment_plan_days',
                                  'plan_list_price', 'actual_amount_paid',
                                  'remaining_plan_duration', 'num_25', 'num_50',
                                  'num_75', 'num_985', 'num_100', 'num_unq',
                                  'total_secs', 'city', 'bd',
                                  'registered_via'])])

In [11]:
X_train_transformed = preprocessor.fit_transform(X)

In [12]:
X_train_transformed = pd.DataFrame(
    X_train_transformed, 
    columns=preprocessor.get_feature_names_out()
)

In [ ]:
X_train_transformed.info()

In [ ]:
#X_train_transformed.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
#X_train_transformed.isnull().sum()

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

def get_estimators():
    return {
        'LogisticRegression': LogisticRegression(),
        'KNN': KNeighborsClassifier(),
        'RandomForest': RandomForestClassifier(),
        'XGB': XGBClassifier()
    }


In [14]:
# cross-validation for each model and store results in a dict
results = []
estimators = get_estimators()
for name, estimator in estimators.items():
    scores = cross_val_score(estimator, X_train_transformed, y, cv=5)
    results.append({
        'Model': name,
        'Mean Accuracy': np.mean(scores),
    })

/Users/andretomaz/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/andretomaz/.pyenv/versions/3.10.6/envs/Churn_prediction/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver op

In [15]:
results

[{'Model': 'LogisticRegression', 'Mean Accuracy': 0.9654217643271089},
 {'Model': 'KNN', 'Mean Accuracy': nan},
 {'Model': 'RandomForest', 'Mean Accuracy': 0.9842711981530632},
 {'Model': 'XGB', 'Mean Accuracy': 0.9845020652395835}]